In [1]:
import xml.etree.ElementTree as ET

# 解析XML文件
tree = ET.parse('enzyme-data.xml')
root = tree.getroot()

# 访问根元素及其属性
print(root.tag)
print(root.attrib)




mysqldump
{}


In [2]:
import xml.etree.ElementTree as ET
import xmltodict

# 解析XML文件
tree = ET.parse('enzyme-data.xml')
root = tree.getroot()

# 将XML数据转换为字典对象
xml_string = ET.tostring(root, encoding='utf8', method='xml')
data_dict = xmltodict.parse(xml_string)

# 访问字典对象及其键值对
print(data_dict['enzymes'])
print(data_dict['enzymes']['enzyme'][0]['common-name'])


ModuleNotFoundError: No module named 'xmltodict'

In [1]:
import sqlite3

# 连接到数据库文件
conn = sqlite3.connect('enzyme-data.sql')

# 创建游标对象
cur = conn.cursor()

# 执行SQL查询语句
cur.execute('SELECT * FROM enzyme')

# 遍历查询结果
for row in cur.fetchall():
    ec_number = row[0]
    common_name = row[1]
    reaction = row[2]
    substrates = row[3]
    products = row[4]
    comments = row[5]

    # 在这里进行进一步处理
    print(f'{ec_number}: {common_name}')
    print(f'Reaction: {reaction}')
    print(f'Substrates: {substrates}')
    print(f'Products: {products}')
    print(f'Comments: {comments}')
    print('---')

# 关闭游标和连接
cur.close()
conn.close()


DatabaseError: file is not a database